In [1]:
%gui qt

In [2]:
import tifffile
import napari
from pathlib import Path
import numpy as np
from tqdm import tqdm
from split_dataset import SplitDataset
from scipy.ndimage import zoom, gaussian_filter
from scipy.ndimage import map_coordinates
import flammkuchen as fl
from dipy.align.imaffine import transform_centers_of_mass, AffineMap
from dipy.align.metrics import CCMetric, EMMetric
from dipy.align.imwarp import SymmetricDiffeomorphicRegistration, DiffeomorphicRegistration, DiffeomorphicMap
from dipy.align.imaffine import (transform_centers_of_mass,
                                 AffineMap,
                                 MutualInformationMetric,
                                 AffineRegistration)
from dipy.align.transforms import (TranslationTransform3D,
                                   RigidTransform3D,
                                   AffineTransform3D)
from fimpy.pipeline.source_extraction import extract_traces

In [3]:
def map_affine(stack,  transform_mat, dest_shape, order=3):
    target_coords = np.reshape(transform_mat @ np.reshape(
        np.pad(np.indices(dest_shape), ((0,1), (0,0), (0,0), (0,0)),
        mode="constant", constant_values=1), (4, -1)), 
                                (3, *dest_shape))

    return map_coordinates(stack, target_coords, order=order)

In [4]:
list_exp = [Path(r"\\FUNES\Shared\Ruben\from MPIN\J-E0041_temperature\ls_1120\7112020_f1_ok\7112020_f1_ok"),
            Path(r"\\FUNES\Shared\Ruben\from MPIN\J-E0041_temperature\ls_1120\7112020_f0_piunochesi_ok\7112020_f0_piunochesi"),
            Path(r"\\FUNES\Shared\Ruben\from MPIN\J-E0041_temperature\ls_1120\6112020_f2_ok\6112020_f2_ok")]

In [106]:
list_exp = list(master_path.glob("*"))

In [5]:
anatomies = []
for exp in list_exp:
    anatomies.append(SplitDataset(exp / "anatomy")[:,:,:])

ValueError: cannot reshape array of size 6600 into shape (24,3,2)

In [109]:
map_rois = []
for exp in list_exp:
    merg_rois = fl.load(exp / "merged_rois.h5")['stack_3D']
    map_rois.append(merg_rois)

In [8]:
v = napari.view_image(anatomies[0])
v.add_image(anatomies[1])
v.add_image(anatomies[2])

<Image layer 'Image [2]' at 0x24c07f6ef88>

In [110]:
reference = anatomies.pop(-1)

In [111]:
reference_rois = map_rois.pop(-1)

In [12]:
# %%time
# morphed_anatomies = []
# morphed_map_rois = []
# transf_matrices = []
# for anatomy in anatomies:
    
#     transform = TranslationTransform3D()

#     translation = affreg.optimize(reference, anatomy, transform, None,
#                                   ref2world, exp2world
#                                   )

#     transform = RigidTransform3D()

#     rigid = affreg.optimize(reference, anatomy, transform, None,
#                                   ref2world, exp2world,
#                                   starting_affine=translation.affine)

# #     transform = AffineTransform3D()

# #     affined = affreg.optimize(reference, anatomy, transform, None,
# #                                   ref2world, exp2world,
# #                                   starting_affine=rigid.affine)

#     morphed_anatomy = rigid.transform(anatomy)
#     morphed_anatomies.append(morphed_anatomy)
#     transf_matrices.append(rigid)

In [13]:
# v = napari.view_image(reference, colormap="green", blending="additive")
# v.add_image(morphed_anatomies[0], colormap="magenta", blending="additive")
# v.add_image(morphed_anatomies[1], colormap="green", blending="additive")

<Image layer 'Image [1]' at 0x1f340f30248>

<Image layer 'Image [1]' at 0x1f340f30248>

<Image layer 'Image [1]' at 0x1f340f30248>

In [157]:
# morphed_roi = (rigid.transform_inverse((reference_rois == 1000).astype(int)) > 0.2).astype(int)

In [13]:
# v = napari.view_image(reference)
# v.add_labels(reference_rois == 1000, blending="additive")
# v.add_labels(morphed_roi, blending="additive")
# # v.add_labels(map_rois[1], blending="additive")

### Clean ROI Label stack

In [30]:
exp = list_exp[2]
reference_rois = fl.load(exp / "merged_rois.h5")['stack_3D']
reference = SplitDataset(exp / "correlation_map")[:,:,:]

In [31]:
v = napari.view_labels(reference_rois)
v.add_image(reference)

<Image layer 'reference' at 0x1b68df7d908>

In [32]:
brain_mask = (v.layers[-1]._data == 0).astype(int)

In [21]:
napari.view_image(brain_mask)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 511.5, 384.5), zoom=1.1849462365591399, angles=(0.0, 0.0, 90.0), interactive=True), cursor=Cursor(position=(0.0, 0.0, 0.0), scaled=True, size=1, style='standard'), dims=Dims(ndim=3, ndisplay=2, last_used=2, range=((0.0, 23.0, 1.0), (0.0, 1023.0, 1.0), (0.0, 769.0, 1.0)), current_step=(0, 0, 0), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(enabled=False, stride=1, shape=(-1, -1)), layers=[<Image layer 'brain_mask' at 0x1b68b9dae08>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position='bottom_right'), active_layer=<Image layer 'brain_mask' at 0x1b68b9dae08>, help='', status='Ready', theme='dark', title='napari', mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x000001B6E6B801F8>], _persisted_mouse_event={}, _mouse_drag_gen={}, _mouse_wheel_gen={}, keymap={})

In [33]:
# fl.save(list_exp[-1] / "brain_mask.h5", brain_mask)
fl.save(exp / "brain_mask.h5", brain_mask)

In [34]:
clean_reference_rois = brain_mask * reference_rois
clean_reference_rois[clean_reference_rois==0] = -1

In [35]:
napari.view_labels(clean_reference_rois)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 510.5, 385.5), zoom=1.1872673849167483, angles=(0.0, 0.0, 90.0), interactive=True), cursor=Cursor(position=(0.0, 0.0, 0.0), scaled=True, size=10, style='standard'), dims=Dims(ndim=3, ndisplay=2, last_used=2, range=((0.0, 22.0, 1.0), (0.0, 1021.0, 1.0), (0.0, 771.0, 1.0)), current_step=(0, 0, 0), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(enabled=False, stride=1, shape=(-1, -1)), layers=[<Labels layer 'clean_reference_rois' at 0x1b68dd567c8>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position='bottom_right'), active_layer=<Labels layer 'clean_reference_rois' at 0x1b68dd567c8>, help='enter paint or fill mode to edit labels', status='Ready', theme='dark', title='napari', mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x000001B6E6B801F8>], _persisted_mouse_event={}, _mouse_drag_gen={}, _m

In [36]:
clean_reference_rois[0,:,:] = -1
clean_reference_rois[-1,:,:] = -1

In [37]:
# fl.save(list_exp[-1] / "roi_to_use.h5", clean_reference_rois)
fl.save(exp / "roi_to_use.h5", clean_reference_rois)

In [191]:
# id_to_transform = list(np.unique(clean_reference_rois))
# roi_canvas = np.zeros(clean_reference_rois.shape)
# for rigid_transf, path_exp in zip (transf_matrices, list_exp[:-1]):
# #     roi_canvas_to_use = roi_canvas.copy()
# #     roi_canvas_to_use_mask = roi_canvas.copy()
# #     for id_ in id_to_transform:
# #         morphed_roi = (rigid_transf.transform_inverse((reference_rois == id_).astype(int)) > 0.2).astype(int)
#     morphed_roi = rigid_transf.transform_inverse(clean_reference_rois, interpolation="nearest")
# #         roi_canvas_to_use_mask += morphed_roi
# #         roi_canvas_to_use += (morphed_roi*id_)
# #         roi_canvas_to_use[roi_canvas_to_use_mask>1] == -1
# #         roi_canvas_to_use_mask[roi_canvas_to_use_mask>1] == -1
# #         if id_ % 200 == 0:
# #             print(id_, "/", id_to_transform[-1])
#     fl.save(path_exp / "roi_to_use.h5", morphed_roi) 

In [198]:
# v = napari.view_labels(clean_reference_rois)
# v.add_labels(morphed_roi)

<Labels layer 'morphed_roi' at 0x1708cb44648>

In [18]:
n_anat = 0

In [19]:
manual_transforms = []

In [20]:
#Open stacks in napari
functional_viewer = napari.view_image(reference)
reference_viewer = napari.view_image(anatomies[n_anat])
#Now we retrieve the points we just defined...

In [21]:
functional_points = functional_viewer.layers[1].data
reference_points = reference_viewer.layers[1].data

X = np.pad(reference_points, ((0,0), (0,1)), mode="constant", constant_values=1)
Y = functional_points

#...and we use some math magic to perform an initial fit 
#(this line performs some sort of lineal regression to find a transform ma
transform_mat =  (np.linalg.pinv(X.T @ X) @ X.T @ Y).T
transformed = map_affine(reference, transform_mat, anatomies[n_anat].shape)

In [22]:
manual_transforms.append(transform_mat)

In [23]:
len(manual_transforms)

1

In [24]:
v = napari.view_image(anatomies[n_anat], colormap="green", blending="additive")
v.add_image(transformed, colormap="magenta", blending="additive")
# v.add_image(morphed_anatomies[1], colormap="green", blending="additive")

<Image layer 'transformed' at 0x2dc14ef7348>

In [25]:
for rigid_transf, path_exp, anatomy_in_loop in zip (manual_transforms, list_exp[:-1], anatomies):
    morphed_roi = map_affine(clean_reference_rois, rigid_transf, anatomy_in_loop.shape,order=0)
    print(path_exp)
    fl.save(path_exp / "roi_to_use.h5", morphed_roi) 

\\FUNES\Shared\experiments\virtual_gradients_experiments\Batch_8(food+flow)\220721_f0\imaging\flow


In [27]:
napari.view_labels(clean_reference_rois)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 339.5, 492.5), zoom=1.2306598984771575, angles=(0.0, 0.0, 90.0), interactive=True), cursor=Cursor(position=(0.0, 0.0, 0.0), scaled=True, size=10, style='standard'), dims=Dims(ndim=3, ndisplay=2, last_used=2, range=((0.0, 22.0, 1.0), (0.0, 679.0, 1.0), (0.0, 985.0, 1.0)), current_step=(0, 0, 0), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(enabled=False, stride=1, shape=(-1, -1)), layers=[<Labels layer 'clean_reference_rois' at 0x2dc15d2b808>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position='bottom_right'), active_layer=<Labels layer 'clean_reference_rois' at 0x2dc15d2b808>, help='enter paint or fill mode to edit labels', status='Ready', theme='dark', title='napari', mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x000002DC0239D288>], _persisted_mouse_event={}, _mouse_drag_gen={}, _mo

In [28]:
rs = fl.load(r"\\FUNES\Shared\experiments\virtual_gradients_experiments\Batch_8(food+flow)\220721_f0\imaging\flow\roi_to_use.h5")

In [30]:
napari.view_labels(rs)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 339.5, 492.5), zoom=1.2306598984771575, angles=(0.0, 0.0, 90.0), interactive=True), cursor=Cursor(position=(0.0, 0.0, 0.0), scaled=True, size=10, style='standard'), dims=Dims(ndim=3, ndisplay=2, last_used=2, range=((0.0, 22.0, 1.0), (0.0, 679.0, 1.0), (0.0, 985.0, 1.0)), current_step=(0, 0, 0), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(enabled=False, stride=1, shape=(-1, -1)), layers=[<Labels layer 'rs' at 0x2dc096146c8>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position='bottom_right'), active_layer=<Labels layer 'rs' at 0x2dc096146c8>, help='enter paint or fill mode to edit labels', status='Ready', theme='dark', title='napari', mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x000002DC0239D288>], _persisted_mouse_event={}, _mouse_drag_gen={}, _mouse_wheel_gen={}, keymap={})

In [33]:
%%time
for exp_path in tqdm(list_exp):
    roi_stack = fl.load(exp_path / "roi_to_use.h5")
    extract_traces(SplitDataset(exp_path / "aligned"), roi_stack, n_jobs=3, block_duration=10)

100%|████████████████████████████████████████████████████████████████████████████████| 3/3 [4:53:23<00:00, 5867.84s/it]


Wall time: 4h 53min 23s


In [65]:
transformed = map_affine(clean_reference_rois, manual_transforms[0], anatomies[0].shape,order=0)

In [72]:
v = napari.view_image(anatomies[0])
v.add_labels(transformed)

<Labels layer 'transformed' at 0x1f33f27dc88>

<Labels layer 'transformed' at 0x1f33f27dc88>

<Labels layer 'transformed' at 0x1f33f27dc88>

In [85]:
result = []
for exp in list_exp:
    result.append(fl.load(exp / "roi_to_use.h5"))

In [86]:
v = napari.view_labels(result[0])
v.add_labels(result[1])
v.add_labels(result[2])

<Labels layer 'Labels [2]' at 0x1f3412a31c8>

<Labels layer 'Labels [2]' at 0x1f3412a31c8>

<Labels layer 'Labels [2]' at 0x1f3412a31c8>